In [ ]:
import numpy as np
import pandas as pd
import datetime as datetime
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt

np.random.seed(1)

# Tech indicator
# Commodity Channel Index
def CCI(data, ndays):
    TP = (data['High'] + data['Low'] + data['Close']) / 3
    CCI = pd.Series((TP - pd.rolling_mean(TP, ndays)) / (0.015 * pd.rolling_std(TP, ndays)),
                    name = 'CCI')
    data = data.join(CCI)
    return data
def RSI(data, ndays):
    delta = data['Close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0
    RolUp = pd.rolling_mean(dUp, ndays)
    RolDown = pd.rolling_mean(dDown, ndays).abs()
    RS = RolUp / RolDown
    RSI = pd.Series((100.0 - (100.0 / (1.0 + RS))),
                    name = 'RSI')
    data = data.join(RSI)
    return data
# Ease of Movement
def EVM(data, ndays):
    dm = ((data['High'] + data['Low'])/2) - ((data['High'].shift(1) + data['Low'].shift(1))/2)
    br = (data['Volume'] / 100000000) / ((data['High'] - data['Low']))
    EVM = dm / br
    EVM_MA = pd.Series(pd.rolling_mean(EVM, ndays), name = 'EVM')
    data = data.join(EVM_MA)
    return data
# Force Index
def ForceIndex(data, ndays):
    FI = pd.Series(data['Close'].diff(ndays) * data['Volume'], name = 'ForceIndex')
    data = data.join(FI)
    return data

def main():

    # Merge tech indicator
    top_20 = ['FB','AAPL','GOOG','INTU','ALGN','AMZN','AAL','MSFT','AMAT','FOX',
          'SBUX','WDC','NFLX','COST','ADBE','EBAY','WDAY','LRCX','BIDU','PYPL']
    n1 = 20
    n2 = 14
    n3 = 14
    n4 = 1
    to_merge = pd.DataFrame({'Date' : [],'CCI': [],'RSI': [],'EVM': [],'Ticker' : [],'ForceIndex':[]})
    for ticker in top_20:
        print('Processing ',ticker)
        data = pdr.get_data_yahoo(ticker, start="2013-01-01", end="2017-12-31")
        data = pd.DataFrame(data)
        data.reset_index(inplace=True)
        Ticker_CCI = CCI(data, n1)
        Ticker_RSI = RSI(Ticker_CCI, n2)
        Ticker_EVM = EVM(Ticker_RSI, n3)
        Ticker_ForceIndex = ForceIndex(Ticker_EVM,n4)
        out = Ticker_ForceIndex[['Date','CCI','RSI','EVM','ForceIndex']]
        out['Ticker']=ticker
        to_merge = pd.concat([to_merge,out])
    to_merge.to_csv('techindicators_20.csv', encoding='utf-8', index=True)